In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI (or Ollama)

Runs locally as an alternative to paid APIs. 

In [ ]:
# Load environment variables in a file called .env
from dotenv import load_dotenv
import os
load_dotenv(override=True)
api_key = os.getenv('OLLAMA_API_KEY')

# Check the key
if not api_key:
    api_key = "ollama"
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.

# Call to the frontier model

In [ ]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hey Llama!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
# A class to represent a Webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://edwarddonner.com") #Thanks to him for the learning material :)
print(ed.title)
print(ed.text)

In [ ]:
# Define our system prompt - you can experiment with this, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

In [ ]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
# calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

## Building useful messages for GPT-4o-mini, using a function

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
print(messages_for(ed))

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2", #Using the free version as an example.
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up.

Also Websites protected with CloudFront (and similar) may give 403 errors.

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")

In [ ]:
# Step 1: Create your prompts

system_prompt = "You are a helpful assistant that summarizes emails consisely."
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt},
           {"role": "user", "content": user_prompt}] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(